In [26]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [ ]:
# 규원님이 전처리 완료하신 최종 데이터 (+라벨까지 붙은)
df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/deeplearning_NLP/perfume/word embedding_hyun/data/dataset_210626_215600.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
len(df)

74779

In [ ]:
df.head()

,gender,name,accords,review,tokenized,only_english,longer_than_2_A,stopwords_removed,lemmatizated,label
0,female,Alien Mugler for women,"['white floral', 'amber', 'woody']","Got a sample of this today, and my 9 year old ...","['got', 'a', 'sample', 'of', 'this', 'today', ...","['got', 'a', 'sample', 'of', 'this', 'today', ...","['got', 'sample', 'this', 'today', 'and', 'yea...","['got', 'sample', 'today', 'year', 'old', 'dau...","['get', 'sample', 'today', 'year', 'old', 'dau...",2.0
1,female,Alien Mugler for women,"['white floral', 'amber', 'woody']",First impressions. Test on blotter.\r\nI have ...,"['first', 'impressions.', 'test', 'on', 'blott...","['first', 'impressions', 'test', 'on', 'blotte...","['first', 'impressions', 'test', 'blotter', 'h...","['first', 'impressions', 'test', 'blotter', 'h...","['first', 'impression', 'test', 'blotter', 'he...",2.0
2,female,Alien Mugler for women,"['white floral', 'amber', 'woody']",This perfume reminds me of my best friend. Act...,"['this', 'perfume', 'reminds', 'me', 'of', 'my...","['this', 'perfume', 'reminds', 'me', 'of', 'my...","['this', 'perfume', 'reminds', 'best', 'friend...","['perfume', 'reminds', 'best', 'friend', 'actu...","['perfume', 'reminds', 'best', 'friend', 'actu...",2.0
3,female,Alien Mugler for women,"['white floral', 'amber', 'woody']",Imagine tripping over your own feet and fallin...,"['imagine', 'tripping', 'over', 'your', 'own',...","['imagine', 'tripping', 'over', 'your', 'own',...","['imagine', 'tripping', 'over', 'your', 'own',...","['imagine', 'tripping', 'feet', 'falling', 'fa...","['imagine', 'trip', 'foot', 'fall', 'face', 'f...",2.0
4,female,Alien Mugler for women,"['white floral', 'amber', 'woody']",Gorgeous Gorgeous Blend ..\r\nLove the scent.....,"['gorgeous', 'gorgeous', 'blend', '..', 'love'...","['gorgeous', 'gorgeous', 'blend', '', 'love', ...","['gorgeous', 'gorgeous', 'blend', 'love', 'the...","['gorgeous', 'gorgeous', 'blend', 'love', 'sce...","['gorgeous', 'gorgeous', 'blend', 'love', 'sce...",2.0


본 데이터는 모두 string 형태라 전처리 필요

In [ ]:
sent_text = df['lemmatizated']

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [38]:
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     break;
print(tokens)

 get sample today year old daughter think smell spray card rotten fish nearly throw immediately however later even decide give fair trial spray crook elbow rot fish smell time get definite bit funk almost urine scent minute blossom gorgeous smooth woody ambery clean warm jasmine daughter didnt even believe tell fragrance smell earlier lol immediately comment fresh clean vibe agree soapy hint powdery way obsess cant stop smelling arm glad give try wait buy full bottle 


In [43]:
import re
from nltk.tokenize import word_tokenize, sent_tokenize

normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     tokens = ' '.join([w for w in tokens.split() if len(w)>=3])
     normalized_text.append(tokens)
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.

result = []
result = [word_tokenize(sentence) for sentence in normalized_text]

In [44]:
for line in result[:3]:
  print(line)

['get', 'sample', 'today', 'year', 'old', 'daughter', 'think', 'smell', 'spray', 'card', 'rotten', 'fish', 'nearly', 'throw', 'immediately', 'however', 'later', 'even', 'decide', 'give', 'fair', 'trial', 'spray', 'crook', 'elbow', 'rot', 'fish', 'smell', 'time', 'get', 'definite', 'bit', 'funk', 'almost', 'urine', 'scent', 'minute', 'blossom', 'gorgeous', 'smooth', 'woody', 'ambery', 'clean', 'warm', 'jasmine', 'daughter', 'didnt', 'even', 'believe', 'tell', 'fragrance', 'smell', 'earlier', 'lol', 'immediately', 'comment', 'fresh', 'clean', 'vibe', 'agree', 'soapy', 'hint', 'powdery', 'way', 'obsess', 'cant', 'stop', 'smelling', 'arm', 'glad', 'give', 'try', 'wait', 'buy', 'full', 'bottle']
['first', 'impression', 'test', 'blotter', 'heard', 'smell', 'like', 'bag', 'spray', 'like', 'gabriella', 'sabatini', 'perfume', 'everybody', 'past', 'woman', 'wear', 'smell', 'addictive', 'plasticky', 'beautiful', 'amaze', 'projection', 'longevity', 'woman', 'strong', 'personality', 'wear', 'may', 

### Word2Vec 훈련시키기

하이퍼 파라미터
* size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.
* window = 컨텍스트 윈도우 크기
* min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)
* workers = 학습을 위한 프로세스 수
* sg = 0은 CBOW, 1은 Skip-gram.

In [45]:
from gensim.models import Word2Vec, KeyedVectors
from gensim.models.callbacks import CallbackAny2Vec
class callback(CallbackAny2Vec): 
  """Callback to print loss after each epoch."""
  def __init__(self):
    self.epoch = 0
    self.loss_to_be_subed = 0
  def on_epoch_end(self, model):
    loss = model.get_latest_training_loss()
    loss_now = loss - self.loss_to_be_subed
    self.loss_to_be_subed = loss
    print('Loss after epoch {}: {}'.format(self.epoch, loss_now))
    self.epoch += 1


model = Word2Vec(sentences=result, size=100, window=10, min_count=2, workers=4, sg=1, compute_loss=True, iter=5, callbacks=[callback()])

Loss after epoch 0: 14180277.0
Loss after epoch 1: 11460299.0
Loss after epoch 2: 9824208.0
Loss after epoch 3: 6196272.0
Loss after epoch 4: 6080840.0


### 5 Epoch 수행 후 결과

In [46]:
model_result = model.wv.most_similar("fruity")
print(model_result)

[('birchy', 0.7936009764671326), ('bubblegumy', 0.7735175490379333), ('citrusy', 0.771584153175354), ('pineappley', 0.7696647047996521), ('sweet', 0.7671349048614502), ('orangey', 0.7593175172805786), ('tart', 0.7578306198120117), ('tarty', 0.754463791847229), ('fruit', 0.7535411715507507), ('citrus', 0.7515233755111694)]


In [48]:
model_result = model.wv.most_similar("aquatic")
print(model_result)

[('marine', 0.8256196975708008), ('oceanic', 0.7870250344276428), ('citrus', 0.7520817518234253), ('fresh', 0.7371219396591187), ('citrusy', 0.7309914231300354), ('acquatic', 0.7304084300994873), ('summery', 0.7289556264877319), ('freshies', 0.7263816595077515), ('melony', 0.7231521606445312), ('ozonic', 0.7149158716201782)]


In [49]:
model_result = model.wv.most_similar("tobacco")
print(model_result)

[('vanille', 0.8515664339065552), ('pipe', 0.7987270355224609), ('cinnamon', 0.7114295959472656), ('honey', 0.7041893601417542), ('chergui', 0.6985769271850586), ('boclet', 0.6752815842628479), ('cinammon', 0.6566763520240784), ('tabac', 0.647133469581604), ('vanilla', 0.6448357105255127), ('hafez', 0.6435563564300537)]


In [50]:
model_result = model.wv.most_similar("floral")
print(model_result)

[('florals', 0.8052598834037781), ('flowery', 0.7894883155822754), ('powdery', 0.7653259038925171), ('lotv', 0.7542228102684021), ('musky', 0.752915620803833), ('whiteairy', 0.7518404722213745), ('citrusgreen', 0.7497655749320984), ('underpin', 0.7491906881332397), ('peachlike', 0.7447079420089722), ('shortlive', 0.7424580454826355)]


In [51]:
model_result = model.wv.most_similar("orange")
print(model_result)

[('blossom', 0.8758841753005981), ('mandarin', 0.8143270015716553), ('mandarine', 0.7933627367019653), ('oranges', 0.7809040546417236), ('creamsicle', 0.7552036046981812), ('grapefruit', 0.7551698684692383), ('peel', 0.749078631401062), ('osmanthus', 0.7396122813224792), ('mimosa', 0.7393823862075806), ('peppercorn', 0.7390335202217102)]


### 모델 저장 및 로드

In [52]:
model.wv.save_word2vec_format('/content/gdrive/MyDrive/Colab Notebooks/deeplearning_NLP/perfume/word embedding_hyun/model/w2v_10window') # 모델 저장

In [53]:
from gensim.models import Word2Vec, KeyedVectors
loaded_model = KeyedVectors.load_word2vec_format('/content/gdrive/MyDrive/Colab Notebooks/deeplearning_NLP/perfume/word embedding_hyun/model/w2v_10window') # 모델 로드

In [54]:
len(loaded_model.vocab)

30054

In [28]:
loaded_model.most_similar("woman")

[('female', 0.7954553961753845),
 ('girl', 0.7846717834472656),
 ('ladys', 0.7796905040740967),
 ('womens', 0.7747268080711365),
 ('heterosexual', 0.7733361124992371),
 ('lady', 0.7628840208053589),
 ('ladies', 0.7533926963806152),
 ('guy', 0.7454804182052612),
 ('kryptonite', 0.7387497425079346),
 ('garnered', 0.7359749674797058)]

In [ ]:
model_result = loaded_model.most_similar("orange")
print(model_result)

[('blossom', 0.8724260330200195), ('mandarin', 0.802692711353302), ('oranges', 0.7801351547241211), ('peel', 0.7669494152069092), ('mandarine', 0.7634068727493286), ('zest', 0.7436349391937256), ('peppercorn', 0.7426453828811646), ('grapefruit', 0.7412831783294678), ('tangerine', 0.7409086227416992), ('mimosa', 0.7328476309776306)]


In [56]:
print(loaded_model.wv.similarity("ocean", "summer"))

0.52681303


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [57]:
print(loaded_model.wv.similarity("forest", "woody"))

0.43691722


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [58]:
print(loaded_model.wv.similarity("jasmine", "floral"))

0.67888105


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [59]:
print(loaded_model.wv.similarity("woman", "feminine"))

0.68685496


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [61]:
print(loaded_model.wv.similarity("ocean", "tobacco"))

0.005945827


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.
